In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense,Flatten,Dropout,Conv2D,MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam
import keras
from keras.models import Sequential
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.metrics import AUC

In [ ]:
train_name=os.listdir('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train')
test_name=os.listdir('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test')

In [ ]:
labels=pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
labels

In [ ]:
sample=pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
sample

In [ ]:
def load_dicom(path):
    dicom=pydicom.read_file(path)
    data=dicom.pixel_array
    data=data-np.min(data)
    if np.max(data) != 0:
        data=data/np.max(data)
    data=(data*255).astype(np.uint8)
    return data

In [ ]:
Img_size = 100
path0='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T1wCE/Image-1.dcm'
img1=load_dicom(path0)
img2=cv2.resize(img1,(Img_size,Img_size)) 
print(img1.shape)
print(img2.shape)

In [ ]:
import random 
ind = []
labels['fold'] = 0
labels0 = labels[labels['MGMT_value']==0]
labels1 = labels[labels['MGMT_value']==1]

while sum(labels0['fold'])<0.2*len(labels0):
    try:
        a = random.randint(0, len(labels0))
        labels0.iloc[a]['fold'] = 1
    except:
        1

while sum(labels1['fold'])<0.2*len(labels1):
    try:
        b = random.randint(0, len(labels1))
        labels1.iloc[b]['fold'] = 1
    except:
        1

labels = pd.concat([labels0, labels1])
labels = labels.sort_values(by = 'BraTS21ID')
labels

sum(labels['fold'])/len(labels)

val_labels = labels[labels['fold']==1]
train_labels = labels[labels['fold']==0]
display(val_labels)
display(train_labels)

In [ ]:
train_dir='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train'

In [ ]:
epochs0 = 100
batch_size = 32

In [ ]:
fld = 'FLAIR'
container = np.load('../input/dataset-with-100100-images-8-for-each-id/trainset_'+fld+'.npz')
trainset_Flair = [container[key] for key in container]     

container = np.load('../input/dataset-with-100100-images-8-for-each-id/trainlabel_'+fld+'.npz')
trainlabel_Flair = [container[key] for key in container]    

container = np.load('../input/dataset-with-100100-images-8-for-each-id/valset_'+fld+'.npz')
valset_Flair = [container[key] for key in container]     

container = np.load('../input/dataset-with-100100-images-8-for-each-id/vallabel_'+fld+'.npz')
vallabel_Flair = [container[key] for key in container]     


In [ ]:
from tensorflow.keras.utils import to_categorical
import copy

trainset = copy.copy(trainset_Flair)
trainlabel = copy.copy(trainlabel_Flair)
valset = copy.copy(valset_Flair)
vallabel = copy.copy(vallabel_Flair)

y0=np.array(trainlabel)
Y_train=to_categorical(y0)
X_train=np.array(trainset)

y1=np.array(vallabel)
Y_val=to_categorical(y1)
X_val=np.array(valset)
# X_test=np.array(testset)


model = Sequential()
model.add(Conv2D(1024,(6,6),input_shape = (Img_size,Img_size,1),activation = 'relu'))
model.add(Conv2D(512,(4,4),activation = 'relu'))
model.add(Dropout(0.2))
model.add(Conv2D(256,(2,2),activation = 'relu'))
model.add(Dropout(0.2))
model.add(Conv2D(128,(2,2),activation = 'relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='softmax'))

from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-6), metrics=['accuracy',AUC()])

callback = EarlyStopping(monitor = 'val_auc', 
                              mode = 'max' , 
                              patience = 15,
                              verbose = 1)

callback2 = ModelCheckpoint(
    filepath="model_Flair.{epoch:02d}-{val_auc:.4f}.h5", 
    monitor='val_auc', mode='max', 
    save_best_only=True, verbose=1
)

display(model.summary())

his = model.fit(X_train, Y_train, validation_data=(X_val,Y_val), epochs=epochs0, batch_size=batch_size, callbacks=[callback,callback2] , verbose=1)



In [ ]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']
get_auc = his.history['auc']
validation_auc = his.history['val_auc']

plt.figure(1,figsize=(20,5))
epochs = range(len(get_acc))
ax0 = plt.subplot(1,3,1)
ax0.plot(epochs, get_acc, 'r', label='Train Accuracy')
ax0.plot(epochs, value_acc, 'b', label='Validation Accuracy')
ax0.legend(loc=0)

epochs = range(len(get_loss))
ax1 = plt.subplot(1,3,2)
ax1.plot(epochs, get_loss, 'r', label='Train Loss')
ax1.plot(epochs, validation_loss, 'b', label='Validation Loss')
ax1.legend(loc=0)


epochs = range(len(get_auc))
ax2 = plt.subplot(1,3,3)
ax2.plot(epochs, get_auc, 'r', label='Training AUC')
ax2.plot(epochs, validation_auc, 'b', label='Validation AUC')
ax2.legend(loc=0)


In [ ]:

fld = 'T1w'
container = np.load('../input/dataset-with-100100-images-8-for-each-id/trainset_'+fld+'.npz')
trainset_T1w = [container[key] for key in container]     

container = np.load('../input/dataset-with-100100-images-8-for-each-id/trainlabel_'+fld+'.npz')
trainlabel_T1w = [container[key] for key in container]    

container = np.load('../input/dataset-with-100100-images-8-for-each-id/valset_'+fld+'.npz')
valset_T1w = [container[key] for key in container]     

container = np.load('../input/dataset-with-100100-images-8-for-each-id/vallabel_'+fld+'.npz')
vallabel_T1w = [container[key] for key in container]     


In [ ]:
from tensorflow.keras.utils import to_categorical
import copy

trainset = copy.copy(trainset_T1w)
trainlabel = copy.copy(trainlabel_T1w)
valset = copy.copy(valset_T1w)
vallabel = copy.copy(vallabel_T1w)

y0=np.array(trainlabel)
Y_train=to_categorical(y0)
X_train=np.array(trainset)

y1=np.array(vallabel)
Y_val=to_categorical(y1)
X_val=np.array(valset)
# X_test=np.array(testset)

del model,callback,callback2,his

model = Sequential()
model.add(Conv2D(1024,(6,6),input_shape = (Img_size,Img_size,1),activation = 'relu'))
model.add(Conv2D(512,(4,4),activation = 'relu'))
model.add(Dropout(0.2))
model.add(Conv2D(256,(2,2),activation = 'relu'))
model.add(Dropout(0.2))
model.add(Conv2D(128,(2,2),activation = 'relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='softmax'))

from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-6), metrics=['accuracy',AUC()])

callback = EarlyStopping(monitor = 'val_auc_1', 
                              mode = 'max' , 
                              patience = 15,
                              verbose = 1)

callback2 = ModelCheckpoint(
    filepath="model_T1w.{epoch:02d}-{val_auc_1:.4f}.h5", 
    monitor='val_auc_1', mode='max', 
    save_best_only=True, verbose=1
)

display(model.summary())

his = model.fit(X_train, Y_train, validation_data=(X_val,Y_val), epochs=epochs0, batch_size=batch_size, callbacks=[callback,callback2] , verbose=1)


In [ ]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']
get_auc = his.history['auc_1']
validation_auc = his.history['val_auc_1']

plt.figure(1,figsize=(20,5))
epochs = range(len(get_acc))
ax0 = plt.subplot(1,3,1)
ax0.plot(epochs, get_acc, 'r', label='Train Accuracy')
ax0.plot(epochs, value_acc, 'b', label='Validation Accuracy')
ax0.legend(loc=0)

epochs = range(len(get_loss))
ax1 = plt.subplot(1,3,2)
ax1.plot(epochs, get_loss, 'r', label='Train Loss')
ax1.plot(epochs, validation_loss, 'b', label='Validation Loss')
ax1.legend(loc=0)


epochs = range(len(get_auc))
ax2 = plt.subplot(1,3,3)
ax2.plot(epochs, get_auc, 'r', label='Training AUC')
ax2.plot(epochs, validation_auc, 'b', label='Validation AUC')
ax2.legend(loc=0)


In [ ]:
fld = 'T1wCE'
container = np.load('../input/dataset-with-100100-images-8-for-each-id/trainset_'+fld+'.npz')
trainset_T1wCE = [container[key] for key in container]     

container = np.load('../input/dataset-with-100100-images-8-for-each-id/trainlabel_'+fld+'.npz')
trainlabel_T1wCE = [container[key] for key in container]    

container = np.load('../input/dataset-with-100100-images-8-for-each-id/valset_'+fld+'.npz')
valset_T1wCE = [container[key] for key in container]     

container = np.load('../input/dataset-with-100100-images-8-for-each-id/vallabel_'+fld+'.npz')
vallabel_T1wCE = [container[key] for key in container]     

In [ ]:
from tensorflow.keras.utils import to_categorical
import copy

trainset = copy.copy(trainset_T1wCE)
trainlabel = copy.copy(trainlabel_T1wCE)
valset = copy.copy(valset_T1wCE)
vallabel = copy.copy(vallabel_T1wCE)

y0=np.array(trainlabel)
Y_train=to_categorical(y0)
X_train=np.array(trainset)

y1=np.array(vallabel)
Y_val=to_categorical(y1)
X_val=np.array(valset)

del model,callback,callback2,his

model = Sequential()
model.add(Conv2D(1024,(6,6),input_shape = (Img_size,Img_size,1),activation = 'relu'))
model.add(Conv2D(512,(4,4),activation = 'relu'))
model.add(Dropout(0.2))
model.add(Conv2D(256,(2,2),activation = 'relu'))
model.add(Dropout(0.2))
model.add(Conv2D(128,(2,2),activation = 'relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='softmax'))

from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-6), metrics=['accuracy',AUC()])

callback = EarlyStopping(monitor = 'val_auc_2', 
                              mode = 'max' , 
                              patience = 15,
                              verbose = 1)

callback2 = ModelCheckpoint(
    filepath="model_T1wCE.{epoch:02d}-{val_auc_2:.4f}.h5", 
    monitor='val_auc_2', mode='max', 
    save_best_only=True, verbose=1
)


display(model.summary())

his = model.fit(X_train, Y_train, validation_data=(X_val,Y_val), epochs=epochs0, batch_size=batch_size, callbacks=[callback,callback2] , verbose=1)

get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']
get_auc = his.history['auc_2']
validation_auc = his.history['val_auc_2']

plt.figure(1,figsize=(20,5))
epochs = range(len(get_acc))
ax0 = plt.subplot(1,3,1)
ax0.plot(epochs, get_acc, 'r', label='Train Accuracy')
ax0.plot(epochs, value_acc, 'b', label='Validation Accuracy')
ax0.legend(loc=0)

epochs = range(len(get_loss))
ax1 = plt.subplot(1,3,2)
ax1.plot(epochs, get_loss, 'r', label='Train Loss')
ax1.plot(epochs, validation_loss, 'b', label='Validation Loss')
ax1.legend(loc=0)


epochs = range(len(get_auc))
ax2 = plt.subplot(1,3,3)
ax2.plot(epochs, get_auc, 'r', label='Training AUC')
ax2.plot(epochs, validation_auc, 'b', label='Validation AUC')
ax2.legend(loc=0)

In [ ]:
fld = 'T2w'
container = np.load('../input/dataset-with-100100-images-8-for-each-id/trainset_'+fld+'.npz')
trainset_T2w = [container[key] for key in container]     

container = np.load('../input/dataset-with-100100-images-8-for-each-id/trainlabel_'+fld+'.npz')
trainlabel_T2w = [container[key] for key in container]    

container = np.load('../input/dataset-with-100100-images-8-for-each-id/valset_'+fld+'.npz')
valset_T2w = [container[key] for key in container]     

container = np.load('../input/dataset-with-100100-images-8-for-each-id/vallabel_'+fld+'.npz')
vallabel_T2w = [container[key] for key in container]     

In [ ]:
from tensorflow.keras.utils import to_categorical
import copy

trainset = copy.copy(trainset_T2w)
trainlabel = copy.copy(trainlabel_T2w)
valset = copy.copy(valset_T2w)
vallabel = copy.copy(vallabel_T2w)

y0=np.array(trainlabel)
Y_train=to_categorical(y0)
X_train=np.array(trainset)

y1=np.array(vallabel)
Y_val=to_categorical(y1)
X_val=np.array(valset)

del model,callback,callback2,his

model = Sequential()
model.add(Conv2D(1024,(6,6),input_shape = (Img_size,Img_size,1),activation = 'relu'))
model.add(Conv2D(512,(4,4),activation = 'relu'))
model.add(Dropout(0.2))
model.add(Conv2D(256,(2,2),activation = 'relu'))
model.add(Dropout(0.2))
model.add(Conv2D(128,(2,2),activation = 'relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='softmax'))

from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-6), metrics=['accuracy',AUC()])

callback = EarlyStopping(monitor = 'val_auc_3', 
                              mode = 'max' , 
                              patience = 15,
                              verbose = 1)

callback2 = ModelCheckpoint(
    filepath="model_T2w.{epoch:02d}-{val_auc_3:.4f}.h5", 
    monitor='val_auc_3', mode='max', 
    save_best_only=True, verbose=1
)

display(model.summary())

his = model.fit(X_train, Y_train, validation_data=(X_val,Y_val), epochs=epochs0, batch_size=batch_size, callbacks=[callback,callback2] , verbose=1)

get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']
get_auc = his.history['auc_3']
validation_auc = his.history['val_auc_3']

plt.figure(1,figsize=(20,5))
epochs = range(len(get_acc))
ax0 = plt.subplot(1,3,1)
ax0.plot(epochs, get_acc, 'r', label='Train Accuracy')
ax0.plot(epochs, value_acc, 'b', label='Validation Accuracy')
ax0.legend(loc=0)

epochs = range(len(get_loss))
ax1 = plt.subplot(1,3,2)
ax1.plot(epochs, get_loss, 'r', label='Train Loss')
ax1.plot(epochs, validation_loss, 'b', label='Validation Loss')
ax1.legend(loc=0)


epochs = range(len(get_auc))
ax2 = plt.subplot(1,3,3)
ax2.plot(epochs, get_auc, 'r', label='Training AUC')
ax2.plot(epochs, validation_auc, 'b', label='Validation AUC')
ax2.legend(loc=0)
